# Spatial Data Science Workshop
## Cristina KADAR & Benjamin RYDER
## 20th September 2017

**Goal:** Familiarize participants with the challenges of working with geo-referenced multi-modal data (governmental data, open data, online digital traces, etc.).

**Problem:** How are an areas Airbnb prices influenced by the listing properties and the attributes of the neighborhood?

**Data:** We will make the data available for download on the tutorial’s website.

**Software:** Participants will be required to have the packages/tools pre-installed. 

## Lets get started..
**Import relevent libraries and check Python version**
We have tested with Python Version: 3.5.2

In [ ]:
%matplotlib inline

from Resources import helpers
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pysal as ps
import geopandas as gpd
# You may need palettable: pip install palettable
from pylab import *
import sys
import platform


print(sys.version_info)
print("Python Version: " + str(platform.python_version()))

**Set some useful parameters**

In [ ]:
# Some useful parameters that we don't need to set later
params = {'legend.fontsize': 20}
plt.rcParams.update(params)
tfs = 20

**And lets define some useful functions which are need later**

In [ ]:
# plotNeighborGraph: Plot maps wiith neighbor links
def plotNeighborGraph(title, grouping):
    f, ax = plt.subplots(1, figsize=(20, 20))
    dataset.plot(ax=ax, linewidth=0.5,color='white')
    plt.plot(centroids[:,0], centroids[:,1],'.')
    for k,neighs in grouping.neighbors.items():
        origin = centroids[k]
        for neigh in neighs:
            segment = centroids[[k,neigh]]
            plt.plot(segment[:,0], segment[:,1], '-')
    plt.title(title, fontsize=tfs)
    ax.set_axis_off()
    show()
    
# analyse_and_transform_wrapper: analyse_and_transform normalises variables with BoxCox
# this function wraps around to use with multiple values  
def analyse_and_transform_wrapper(names,data):
    for n in names:
        data[n] = helpers.analyse_and_transform(data[n])
    return data

## Data
**We need to import the data that we're using - this should be included in the git project**

In [ ]:
shp_link = "./Data_Part2/nyc_census_tract_features_v3_export.shp"
dataset =


# Interested in the full list of variables? Uncomment and run the command below
#dataset.columns.tolist()

**Lets see what area this data looks like!**

In [ ]:
f, ax = plt.subplots(1, figsize=(20, 20))

plt.title('', fontsize=tfs)
ax.set_axis_off()
plt.show()

**Let's also generate the central points for each of these census tracts...**

In [ ]:
dataframe = 
centroids = 

**... And display these on the same map as before**

## Neighbors
**Neighbors can be defined in many ways, one of the simplest is to take the** 'K-nearest Neighbors' **of each centroid point**

**This is very easy with pysal and shapefiles!**

In [ ]:
knn = 


**To test this, we can take a single point, and link this with its neighbors**

In [ ]:
self_and_neighbors =




**Lets use one of the predefined functions to visualise this to check how the linking looks**

**There are clearly some issues with this approach**

**Other techniques are more suitabe when the map data is available**

**Queens Neighbors are those that share an edge OR a corner, and can also be easily generated from shapefiles** 

In [ ]:
qW = 


**This is much better, but perhaps overkill for our problem right now.**

**Especially as US census tracts are often 'grid' based.**

** Rook Neighbors are similar to Queens, but link only via edges (and not corners)**

**.. and can be generated in the same way as before**

In [ ]:
rW = 


## Visualising the data

** So we have some neighbors, but what are we actually investigating here**

** Average AirBnB listing price in New York Census Tracts**

** Lets add this to our map!**

In [ ]:
f, ax = plt.subplots(1, figsize=(20, 20))

plt.title('', fontsize=tfs)
ax.set_axis_off()
plt.show()

**Interesting, but not so useful for us to interpret really**

**We should take a closer look at the data**

**There are some clear outliers**

**We can manage these outliers by calculating the quantiles of the data, and help us see patterns in the data**

In [ ]:
bin_n=
quantiles = 


**Let's see if this helps us interpret the data more easily..**

In [ ]:
f, ax = plt.subplots(1, figsize=(20, 20))





plt.title('', fontsize=tfs)
ax.set_axis_off()
plt.show()

**Interesting, but still not very intuitive or easy to interpret**

** Color maps can be found at following link: **
https://matplotlib.org/examples/color/colormaps_reference.html

In [ ]:
f, ax = plt.subplots(1, figsize=(20, 20))
dataset.assign( = ).plot(ax= , \
                                                linewidth= , \
                                                column= , \
                                                cmap= , \
                                                categorical= , \
                                                legend= )

plt.title('', fontsize=tfs)
ax.set_axis_off()
plt.show()

## Regressions

**We can 'see' some spatial patterns in the data, so let's get down to the regression business** 

** First step: Normalising our dependent variable**

In [ ]:
name_y = 
y = 
ln_y = 
transformed_y = 

# An alternative is to use the BoxCox transformation, uncomment the line below to apply this instead
#transformed_y = analyse_and_transform_wrapper(name_y,y)

**We'll use the Rook Neighbors weighting from before**

**The weights have to be transformed into rows in order to be used...**

**... and then use the pysal spatial lag function with the weights and our dependent variable**

In [ ]:
rW.transform = 
transformed_y_lag = 

**Let's see what these lag values look like!**

In [ ]:
f, ax = plt.subplots(1, figsize=(20, 20))
dataset.assign( = ).plot(ax= , \
                                                         linewidth= , \
                                                         column= , \
                                                         cmap= )

plt.title('', fontsize=tfs)
ax.set_axis_off()
plt.show()

**In PySAL, commonly-used analysis methods are very easy to access. **

**For example, we can quickly compute a Moran's I statistic for price**

In [ ]:
MI_transformed_y = 


**We can plot this and see the relationship between the price and the surrounding lagged price** 

In [ ]:
f, ax = plt.subplots(1, figsize=(9, 9))

plt.plot( ,  , '.', color='firebrick')

plt.vlines(transformed_y.mean(), transformed_y_lag.min(), transformed_y_lag.max(), linestyle='--')
plt.hlines(transformed_y_lag.mean(), transformed_y.min(), transformed_y.max(), linestyle='--')

# red line of best fit using global I as slope
plt.plot( , * , 'r')

plt.title(' ')
plt.ylabel('Spatial Lag of Transformed Variable')
plt.xlabel('Transformed Variable')
plt.show()


**Lets grab some independent variables...**

In [ ]:
name_x = 
x =  
transformed_x = 

**To run the model, we can use the spreg module in PySAL, which implements a standard OLS routine, but is particularly well suited for regressions on spatial data. Also, although for the initial model we do not need it, let us build a spatial weights matrix that connects every observation to its 8 nearest neighbors. This will allow us to get extra diagnostics from the baseline model.**

In [ ]:
model_ols = ps.spreg.OLS(y= , \
                         x= , \
                         w= , \
                         name_y= , \
                         name_x= , \
                         spat_diag= )


**If we include the lagged prices of the neighbors in the model, we violate some of the assumptions on which OLS relies. **

**We can use GM_lag to account for this and include these spatially lagged varibles**

In [ ]:
model_lag = ps.spreg.GM_Lag(y=, \
                            x=, \
                            w=, \
                            name_y=, \
                            name_x=)


**There are several over spatial regression methods built into PySAL**

**Spatial Error is when the error values in the models are spatially lagged**

In [ ]:
model_error = ps.spreg.GM_Error(y= , \
                                x= , \
                                w= , \
                                name_y= , \
                                name_x= )


**Spatial Combo is when both the dependent variable and the error values in the models are spatially lagged**

In [ ]:
model_combo = ps.spreg.GM_Combo(y= , \
                                x= , \
                                w= , \
                                name_y= , \
                                name_x= )


**Interested in trying out some other regressions? Here's a list of the variables in the dataset**

*Needs interpretting..*

In [ ]:
dataset.columns.tolist()